In [ ]:
import os
import json
import pickle
import re
from collections import defaultdict, Counter
from typing import List, Dict, Set, Tuple
import math

class NGramSearchSystem:
    def __init__(self, corpus_folder: str, max_ngram_size: int = 5):
        """
        Initialize N-gram Search System up to specified N-gram size
        
        Args:
            corpus_folder: Path to the cleaned_corpus folder
            max_ngram_size: Maximum N for N-grams (2=bigram, 3=trigram, 4=4-gram, 5=5-gram)
        """
        self.corpus_folder = corpus_folder
        self.max_ngram_size = max_ngram_size
        self.index_folder = os.path.join(corpus_folder, f"ngram_index_{max_ngram_size}")
        
        # Create index folder if it doesn't exist
        if not os.path.exists(self.index_folder):
            os.makedirs(self.index_folder)
        
        # Data structures
        self.documents = {}  # doc_id -> document info
        self.doc_tokens = {}  # doc_id -> list of tokens
        self.ngram_index = defaultdict(lambda: defaultdict(set))  # n -> ngram -> doc_ids
        self.position_index = defaultdict(lambda: defaultdict(dict))  # n -> doc_id -> ngram -> positions
        
        # Statistics
        self.stats = {
            'total_documents': 0,
            'total_ngrams': defaultdict(int),
            'unique_ngrams': defaultdict(int),
            'avg_ngrams_per_doc': defaultdict(float)
        }
    
    def extract_ngrams(self, tokens: List[str], n: int) -> List[Tuple[str, int]]:
        """
        Extract N-grams from tokens with their positions
        
        Args:
            tokens: List of tokens
            n: N-gram size (2 for bigram, 3 for trigram, etc.)
        
        Returns:
            List of (ngram_text, start_position)
        """
        ngrams = []
        # Allow overlap by moving window by 1 token each time
        for i in range(len(tokens) - n + 1):
            ngram = " ".join(tokens[i:i+n])
            ngrams.append((ngram, i))
        return ngrams
    
    def build_index(self):
        """Build N-gram index from corpus up to max_ngram_size"""
        print(f"🔨 Building N-gram Search Index (up to {self.max_ngram_size}-grams)...")
        
        # Load document tokens
        doc_tokens_file = os.path.join(self.corpus_folder, "document_tokens.json")
        
        if not os.path.exists(doc_tokens_file):
            print("❌ document_tokens.json not found!")
            print("Looking for document tokens in alternative locations...")
            
            # Try to find document_tokens.json
            for root, dirs, files in os.walk(self.corpus_folder):
                if "document_tokens.json" in files:
                    doc_tokens_file = os.path.join(root, "document_tokens.json")
                    print(f"✅ Found at: {doc_tokens_file}")
                    break
            
            if not os.path.exists(doc_tokens_file):
                print("❌ Could not find document_tokens.json")
                return False
        
        try:
            with open(doc_tokens_file, 'r', encoding='utf-8') as f:
                doc_data = json.load(f)
            print(f"✅ Loaded {len(doc_data)} documents")
        except Exception as e:
            print(f"❌ Error loading document tokens: {e}")
            return False
        
        # Process each document
        doc_id = 0
        total_docs = len(doc_data)
        
        for doc_name, doc_info in doc_data.items():
            doc_id += 1
            tokens = doc_info.get('tokens', [])
            token_count = doc_info.get('token_count', 0)
            
            if doc_id % 100 == 0:
                print(f"   Processing document {doc_id}/{total_docs}...")
            
            if tokens and token_count >= 2:  # Need at least 2 tokens for bigrams
                doc_key = f"doc_{doc_id:05d}"  # Format as doc_00001
                
                # Store document info
                self.documents[doc_key] = {
                    'name': doc_name,
                    'token_count': token_count
                }
                
                # Store tokens
                self.doc_tokens[doc_key] = tokens
                
                # Extract N-grams for each size (2 to max_ngram_size)
                for n in range(2, self.max_ngram_size + 1):
                    if len(tokens) >= n:  # Only extract if document has enough tokens
                        ngrams = self.extract_ngrams(tokens, n)
                        
                        # Update statistics
                        self.stats['total_ngrams'][n] += len(ngrams)
                        
                        # Add to index
                        positions = defaultdict(list)
                        for ngram_text, position in ngrams:
                            # Add to ngram index
                            self.ngram_index[n][ngram_text].add(doc_key)
                            
                            # Store position
                            positions[ngram_text].append(position)
                        
                        # Store positions
                        if positions:
                            self.position_index[n][doc_key] = positions
        
        # Update statistics
        self.stats['total_documents'] = len(self.documents)
        
        # Calculate unique N-grams
        for n in range(2, self.max_ngram_size + 1):
            if n in self.ngram_index:
                self.stats['unique_ngrams'][n] = len(self.ngram_index[n])
                if self.stats['total_documents'] > 0:
                    self.stats['avg_ngrams_per_doc'][n] = self.stats['total_ngrams'][n] / self.stats['total_documents']
        
        # Save index
        self.save_index()
        
        print(f"\n✅ N-gram index built successfully!")
        print(f"   Documents indexed: {self.stats['total_documents']:,}")
        print(f"   Document tokens file: {doc_tokens_file}")
        
        for n in range(2, self.max_ngram_size + 1):
            if n in self.stats['unique_ngrams']:
                print(f"   {n}-grams: {self.stats['unique_ngrams'][n]:,} unique, "
                      f"{self.stats['avg_ngrams_per_doc'][n]:.1f} avg per doc")
        
        # Show some sample N-grams
        print(f"\n📊 Sample N-grams in index:")
        for n in range(2, min(4, self.max_ngram_size + 1)):  # Show up to trigrams
            if n in self.ngram_index and self.ngram_index[n]:
                sample_ngrams = list(self.ngram_index[n].keys())[:5]
                print(f"   {n}-grams: {', '.join(sample_ngrams)}...")
        
        return True
    
    def save_index(self):
        """Save index to disk"""
        # Convert sets to lists for serialization
        index_data = {
            'documents': self.documents,
            'doc_tokens': self.doc_tokens,
            'ngram_index': {
                n: {k: list(v) for k, v in index.items()}
                for n, index in self.ngram_index.items()
            },
            'position_index': {
                n: {
                    doc_id: dict(positions)
                    for doc_id, positions in index.items()
                }
                for n, index in self.position_index.items()
            },
            'stats': self.stats,
            'max_ngram_size': self.max_ngram_size
        }
        
        index_file = os.path.join(self.index_folder, "ngram_index.pkl")
        with open(index_file, 'wb') as f:
            pickle.dump(index_data, f)
        
        # Also save a human-readable version
        readable_file = os.path.join(self.index_folder, "index_stats.json")
        readable_data = {
            'stats': self.stats,
            'max_ngram_size': self.max_ngram_size,
            'sample_ngrams': {
                n: list(self.ngram_index[n].keys())[:20] if n in self.ngram_index else []
                for n in range(2, self.max_ngram_size + 1)
            }
        }
        with open(readable_file, 'w', encoding='utf-8') as f:
            json.dump(readable_data, f, indent=2, ensure_ascii=False)
        
        print(f"\n💾 Index saved to: {index_file}")
        print(f"📊 Statistics saved to: {readable_file}")
    
    def load_index(self):
        """Load index from disk"""
        index_file = os.path.join(self.index_folder, "ngram_index.pkl")
        
        if not os.path.exists(index_file):
            print(f"❌ Index not found at: {index_file}")
            print("Building new index...")
            return self.build_index()
        
        try:
            print(f"📂 Loading index from: {index_file}")
            with open(index_file, 'rb') as f:
                index_data = pickle.load(f)
            
            self.documents = index_data['documents']
            self.doc_tokens = index_data['doc_tokens']
            self.max_ngram_size = index_data.get('max_ngram_size', 5)
            self.stats = index_data['stats']
            
            # Reconstruct ngram_index
            self.ngram_index = defaultdict(lambda: defaultdict(set))
            for n, index in index_data['ngram_index'].items():
                n_int = int(n)
                for ngram, doc_ids in index.items():
                    self.ngram_index[n_int][ngram] = set(doc_ids)
            
            # Reconstruct position_index
            self.position_index = defaultdict(lambda: defaultdict(dict))
            for n, index in index_data['position_index'].items():
                n_int = int(n)
                for doc_id, positions in index.items():
                    self.position_index[n_int][doc_id] = positions
            
            print(f"✅ N-gram index loaded successfully!")
            print(f"   Documents: {self.stats['total_documents']:,}")
            print(f"   Max N-gram size: {self.max_ngram_size}")
            
            for n in range(2, self.max_ngram_size + 1):
                if n in self.stats['unique_ngrams']:
                    print(f"   {n}-grams: {self.stats['unique_ngrams'][n]:,} unique")
            
            return True
        except Exception as e:
            print(f"❌ Error loading index: {e}")
            import traceback
            traceback.print_exc()
            return self.build_index()
    
    def find_best_ngram_size(self, query_tokens: List[str]) -> int:
        """
        Find the best N-gram size for the query
        
        Returns the largest N where:
        1. N <= query length
        2. N <= max_ngram_size
        3. There are documents containing the N-grams
        """
        query_length = len(query_tokens)
        
        # Try from largest to smallest for better precision
        for n in range(min(self.max_ngram_size, query_length), 1, -1):
            if query_length >= n:
                # Check if any documents contain the first N-gram
                first_ngram = " ".join(query_tokens[:n])
                if first_ngram in self.ngram_index[n]:
                    return n
        
        # Fall back to bigram if nothing else works
        return 2
    
    def exact_phrase_search(self, phrase: str) -> List[Dict]:
        """
        Exact phrase search using optimal N-gram size
        
        Args:
            phrase: Search phrase (e.g., "supreme court of pakistan")
        
        Returns:
            List of matching documents with context
        """
        if not phrase or not phrase.strip():
            print("❌ Empty query")
            return []
        
        # Clean and tokenize the query
        query_tokens = phrase.lower().strip().split()
        query_length = len(query_tokens)
        
        if query_length < 2:
            print("❌ Phrase must contain at least 2 words")
            return []
        
        print(f"\n🔍 Searching for phrase: '{phrase}'")
        print(f"   Query tokens ({query_length}): {query_tokens}")
        
        # Find optimal N-gram size
        optimal_n = self.find_best_ngram_size(query_tokens)
        print(f"   Using {optimal_n}-gram matching")
        
        # Extract query N-grams
        query_ngrams = self.extract_ngrams(query_tokens, optimal_n)
        query_ngram_texts = [ngram for ngram, _ in query_ngrams]
        
        print(f"   Query {optimal_n}-grams ({len(query_ngrams)}): {query_ngram_texts[:5]}")
        if len(query_ngrams) > 5:
            print(f"   ... and {len(query_ngrams) - 5} more")
        
        # Step 1: Find candidate documents containing ALL query N-grams
        candidate_docs = None
        ngram_doc_counts = []
        
        for ngram_text, _ in query_ngrams:
            docs_with_ngram = self.ngram_index[optimal_n].get(ngram_text, set())
            ngram_doc_counts.append(len(docs_with_ngram))
            
            if candidate_docs is None:
                candidate_docs = docs_with_ngram.copy()
            else:
                candidate_docs = candidate_docs.intersection(docs_with_ngram)
            
            if not candidate_docs:
                print(f"   ✗ No documents contain all {optimal_n}-grams")
                print(f"   N-gram frequency: {ngram_doc_counts}")
                return []
        
        print(f"   ✓ Candidate documents: {len(candidate_docs)}")
        print(f"   N-gram frequencies: {ngram_doc_counts}")
        
        # Step 2: Verify phrase occurrence in candidate documents
        results = []
        for doc_id in candidate_docs:
            doc_info = self.documents[doc_id]
            tokens = self.doc_tokens[doc_id]
            
            # Find all positions where the full phrase occurs
            phrase_positions = self.find_phrase_positions_direct(
                tokens, query_tokens
            )
            
            if phrase_positions:
                # Calculate match quality score
                match_score = self.calculate_match_score(
                    phrase_positions, query_length, len(tokens)
                )
                
                # Get context around matches
                contexts = []
                for pos in phrase_positions[:2]:  # Limit to first 2 matches
                    context = self.get_context(tokens, pos, query_length)
                    contexts.append(context)
                
                results.append({
                    'doc_id': doc_id,
                    'name': doc_info['name'],
                    'token_count': doc_info['token_count'],
                    'match_score': match_score,
                    'match_count': len(phrase_positions),
                    'positions': phrase_positions[:3],  # First 3 positions
                    'contexts': contexts,
                    'matched_phrase': phrase
                })
        
        # Sort by match score (descending)
        results.sort(key=lambda x: x['match_score'], reverse=True)
        
        print(f"   ✓ Documents with exact phrase: {len(results)}")
        return results
    
    def find_phrase_positions_direct(self, tokens: List[str], 
                                    query_tokens: List[str]) -> List[int]:
        """
        Direct search for phrase positions (slower but accurate)
        
        Returns:
            List of start positions where phrase occurs
        """
        phrase_text = " ".join(query_tokens)
        tokens_text = " ".join(tokens)
        
        # Simple but effective: find all occurrences
        positions = []
        query_length = len(query_tokens)
        
        for i in range(len(tokens) - query_length + 1):
            if tokens[i:i+query_length] == query_tokens:
                positions.append(i)
        
        return positions
    
    def calculate_match_score(self, positions: List[int], 
                            query_length: int, doc_length: int) -> float:
        """
        Calculate match quality score
        
        Higher score for:
        1. More occurrences of the phrase
        2. Shorter documents (relative importance)
        3. Phrases in document beginning (higher relevance)
        """
        if not positions or doc_length == 0:
            return 0.0
        
        # Base score based on number of occurrences
        occurrence_score = min(len(positions) * 15, 60)
        
        # Position score (earlier occurrences are better)
        position_score = 0
        for pos in positions[:3]:  # Consider first 3 occurrences
            # Normalize position (0 = beginning, 1 = end)
            normalized_pos = 1.0 - (pos / max(doc_length, 1))
            position_score += normalized_pos * 15
        
        # Density score (phrase density in document)
        density = (len(positions) * query_length) / max(doc_length, 1)
        density_score = min(density * 120, 40)
        
        # Combine scores
        total_score = occurrence_score + position_score + density_score
        
        # Normalize to 0-100 scale
        return min(total_score, 100.0)
    
    def get_context(self, tokens: List[str], position: int, 
                   phrase_length: int, window: int = 7) -> str:
        """Get context around the phrase match"""
        start = max(0, position - window)
        end = min(len(tokens), position + phrase_length + window)
        
        # Build context with highlighting
        context_parts = []
        for i in range(start, end):
            if i == position:
                context_parts.append("[")
            context_parts.append(tokens[i])
            if i == position + phrase_length - 1:
                context_parts.append("]")
        
        return " ".join(context_parts)
    
    def multi_size_ngram_search(self, phrase: str) -> List[Dict]:
        """
        Search using multiple N-gram sizes for better recall
        
        Args:
            phrase: Search phrase
        
        Returns:
            Combined results from different N-gram sizes
        """
        query_tokens = phrase.lower().strip().split()
        query_length = len(query_tokens)
        
        if query_length < 2:
            return []
        
        print(f"\n🔍 Multi-size N-gram search: '{phrase}'")
        
        all_results = []
        
        # Try different N-gram sizes
        for n in range(min(self.max_ngram_size, query_length), 1, -1):
            if query_length >= n:
                print(f"   Trying {n}-gram search...")
                
                # Extract N-grams
                ngrams = self.extract_ngrams(query_tokens, n)
                ngram_texts = [ngram for ngram, _ in ngrams]
                
                # Find documents containing any of these N-grams
                docs_found = set()
                for ngram_text in ngram_texts:
                    docs = self.ngram_index[n].get(ngram_text, set())
                    docs_found.update(docs)
                
                if docs_found:
                    print(f"   ✓ Found {len(docs_found)} documents with {n}-grams")
                    
                    # Score documents based on N-gram coverage
                    for doc_id in docs_found:
                        doc_info = self.documents[doc_id]
                        
                        # Count how many query N-grams appear in document
                        ngram_count = 0
                        for ngram_text in ngram_texts:
                            if doc_id in self.ngram_index[n].get(ngram_text, set()):
                                ngram_count += 1
                        
                        # Calculate coverage score
                        coverage = ngram_count / len(ngram_texts)
                        score = coverage * 100 * (n / query_length)  # Weight by N-gram size
                        
                        # Check for exact phrase
                        tokens = self.doc_tokens[doc_id]
                        exact_positions = self.find_phrase_positions_direct(
                            tokens, query_tokens
                        )
                        
                        if exact_positions:
                            score *= 1.5  # Bonus for exact match
                        
                        all_results.append({
                            'doc_id': doc_id,
                            'name': doc_info['name'],
                            'token_count': doc_info['token_count'],
                            'match_score': score,
                            'ngram_size': n,
                            'coverage': coverage,
                            'exact_match': len(exact_positions) > 0,
                            'match_count': len(exact_positions) if exact_positions else 0
                        })
                else:
                    print(f"   ✗ No documents found with {n}-grams")
        
        # Remove duplicates and sort
        unique_results = {}
        for result in all_results:
            doc_id = result['doc_id']
            if doc_id not in unique_results or result['match_score'] > unique_results[doc_id]['match_score']:
                unique_results[doc_id] = result
        
        final_results = list(unique_results.values())
        final_results.sort(key=lambda x: x['match_score'], reverse=True)
        
        print(f"   ✓ Total unique documents: {len(final_results)}")
        return final_results
    
    def search_with_suggestions(self, phrase: str) -> Tuple[List[Dict], List[str]]:
        """
        Search with spelling suggestions and query expansion
        
        Returns:
            (results, suggestions)
        """
        original_phrase = phrase
        query_tokens = phrase.lower().strip().split()
        
        # First try exact search
        results = self.exact_phrase_search(phrase)
        
        if results:
            return results, ["Exact phrase match found"]
        
        # If no results, try multi-size search
        print("\n🤔 No exact matches found, trying multi-size N-gram search...")
        results = self.multi_size_ngram_search(phrase)
        
        suggestions = []
        
        if not results:
            # Generate suggestions
            suggestions.append("No documents found. Try:")
            suggestions.append("1. Check spelling of your phrase")
            suggestions.append("2. Try a shorter phrase")
            suggestions.append("3. Try individual important words")
            
            # Suggest similar phrases based on common N-grams
            if len(query_tokens) >= 2:
                # Look for documents containing parts of the phrase
                for n in range(2, min(4, len(query_tokens) + 1)):
                    ngrams = self.extract_ngrams(query_tokens, n)
                    for ngram_text, _ in ngrams[:3]:  # Check first 3 N-grams
                        if ngram_text in self.ngram_index[n]:
                            docs_count = len(self.ngram_index[n][ngram_text])
                            suggestions.append(f"Found '{ngram_text}' in {docs_count} documents")
        
        return results, suggestions
    
    def show_document_ngrams(self, doc_name: str, n: int = 3, limit: int = 20):
        """Show N-grams for a specific document"""
        # Find document
        doc_id = None
        for d_id, info in self.documents.items():
            if info['name'] == doc_name:
                doc_id = d_id
                break
        
        if not doc_id:
            print(f"❌ Document not found: {doc_name}")
            return
        
        if n not in self.position_index or doc_id not in self.position_index[n]:
            print(f"❌ No {n}-grams found for document {doc_name}")
            return
        
        print(f"\n🔤 {n}-GRAMS IN DOCUMENT: {doc_name}")
        print("=" * 80)
        
        ngrams_info = self.position_index[n][doc_id]
        sorted_ngrams = sorted(ngrams_info.items(), key=lambda x: len(x[1]), reverse=True)
        
        print(f"Total {n}-grams: {len(ngrams_info)}\n")
        
        for i, (ngram, positions) in enumerate(sorted_ngrams[:limit], 1):
            freq = len(positions)
            sample_positions = positions[:3] if len(positions) > 3 else positions
            print(f"{i:3d}. {ngram}")
            print(f"     Frequency: {freq}, Positions: {sample_positions}")
            if len(positions) > 3:
                print(f"     ... and {len(positions) - 3} more positions")
            print()
        
        if len(sorted_ngrams) > limit:
            print(f"... and {len(sorted_ngrams) - limit} more {n}-grams")
        
        print("=" * 80)
    
    def interactive_search(self):
        """Interactive N-gram search interface"""
        print("\n" + "=" * 80)
        print("🔤 N-GRAM PHRASE SEARCH SYSTEM")
        print(f"   Max N-gram size: {self.max_ngram_size}")
        print("=" * 80)
        print("\n📋 Available Commands:")
        print("  • search <phrase>     - Exact phrase search")
        print("  • multi <phrase>      - Multi-size N-gram search")
        print("  • smart <phrase>      - Smart search with suggestions")
        print("  • ngrams <doc> <n>    - Show N-grams for document")
        print("  • stats               - Show index statistics")
        print("  • example             - Show example searches")
        print("  • quit                - Exit")
        print("=" * 80)
        
        while True:
            user_input = input("\n🎯 Enter command: ").strip()
            
            if not user_input:
                continue
            
            if user_input.lower() == 'quit':
                print("👋 Goodbye!")
                break
            
            elif user_input.lower() == 'stats':
                self.show_statistics()
            
            elif user_input.lower() == 'example':
                self.show_examples()
            
            elif user_input.lower().startswith('search '):
                phrase = user_input[7:].strip()
                if phrase:
                    results = self.exact_phrase_search(phrase)
                    self.display_results(results, f"Exact phrase: '{phrase}'")
                else:
                    print("❌ Please enter a search phrase")
            
            elif user_input.lower().startswith('multi '):
                phrase = user_input[6:].strip()
                if phrase:
                    results = self.multi_size_ngram_search(phrase)
                    self.display_results(results, f"Multi-size: '{phrase}'")
                else:
                    print("❌ Please enter a search phrase")
            
            elif user_input.lower().startswith('smart '):
                phrase = user_input[6:].strip()
                if phrase:
                    results, suggestions = self.search_with_suggestions(phrase)
                    if results:
                        self.display_results(results, f"Smart search: '{phrase}'")
                    else:
                        print(f"\n❌ No results found for '{phrase}'")
                        if suggestions:
                            print("\n💡 Suggestions:")
                            for suggestion in suggestions:
                                print(f"   • {suggestion}")
                else:
                    print("❌ Please enter a search phrase")
            
            elif user_input.lower().startswith('ngrams '):
                parts = user_input[7:].strip().split()
                if len(parts) >= 1:
                    doc_name = parts[0]
                    n = int(parts[1]) if len(parts) > 1 else 3
                    if 2 <= n <= self.max_ngram_size:
                        self.show_document_ngrams(doc_name, n)
                    else:
                        print(f"❌ N must be between 2 and {self.max_ngram_size}")
                else:
                    print("❌ Usage: ngrams <document_name> [n]")
            
            else:
                # Default to smart search
                results, suggestions = self.search_with_suggestions(user_input)
                if results:
                    self.display_results(results, f"Search: '{user_input}'")
                else:
                    print(f"\n❌ No results found for '{user_input}'")
                    if suggestions:
                        print("\n💡 Suggestions:")
                        for suggestion in suggestions:
                            print(f"   • {suggestion}")
    
    def show_statistics(self):
        """Show detailed index statistics"""
        print(f"\n📊 N-GRAM INDEX STATISTICS")
        print("=" * 80)
        print(f"Documents indexed: {self.stats['total_documents']:,}")
        print(f"Max N-gram size: {self.max_ngram_size}")
        print("-" * 80)
        
        for n in range(2, self.max_ngram_size + 1):
            if n in self.stats['unique_ngrams']:
                unique = self.stats['unique_ngrams'][n]
                total = self.stats['total_ngrams'][n]
                avg = self.stats['avg_ngrams_per_doc'][n]
                print(f"{n}-grams: {unique:,} unique, {total:,} total, {avg:.1f} avg/doc")
        
        # Show sample of most common N-grams
        print("-" * 80)
        print("📈 MOST COMMON N-GRAMS:")
        
        for n in range(2, min(4, self.max_ngram_size + 1)):
            if n in self.ngram_index and self.ngram_index[n]:
                # Get N-grams sorted by document frequency
                ngram_freqs = [(ngram, len(docs)) for ngram, docs in self.ngram_index[n].items()]
                ngram_freqs.sort(key=lambda x: x[1], reverse=True)
                
                print(f"\nTop {n}-grams:")
                for i, (ngram, freq) in enumerate(ngram_freqs[:10], 1):
                    percentage = (freq / self.stats['total_documents']) * 100
                    print(f"  {i:2d}. {ngram:<30s} {freq:4d} docs ({percentage:.1f}%)")
        
        print("=" * 80)
    
    def show_examples(self):
        """Show example searches"""
        print("\n📝 EXAMPLE SEARCHES")
        print("=" * 80)
        print("\nShort phrases (2-3 words):")
        print("  • supreme court")
        print("  • murder evidence")
        print("  • court appeal")
        print("  • criminal case")
        
        print("\nMedium phrases (4-5 words):")
        print("  • supreme court of pakistan")
        print("  • high court appeal case")
        print("  • evidence of murder weapon")
        print("  • constitutional right violation")
        
        print("\nLonger phrases:")
        print("  • in the matter of criminal appeal")
        print("  • court of appeal decision")
        print("  • murder case evidence hearing")
        
        print("\n💡 Tips:")
        print("  1. Use exact phrases for precise matching")
        print("  2. Longer phrases give more specific results")
        print("  3. Try different phrase lengths if no results")
        print("=" * 80)
    
    def display_results(self, results: List[Dict], title: str):
        """Display search results"""
        if not results:
            print(f"\n❌ No results found")
            return
        
        print(f"\n✅ {title}")
        print(f"📊 Found {len(results)} document(s)")
        print("=" * 80)
        
        for i, result in enumerate(results[:15], 1):  # Show first 15
            print(f"\n{i:2d}. 📄 {result['name']}")
            print(f"    📏 Length: {result['token_count']:,} tokens")
            print(f"    ⭐ Score: {result['match_score']:.1f}")
            
            if 'match_count' in result and result['match_count'] > 0:
                print(f"    🔍 Exact matches: {result['match_count']}")
            
            if 'coverage' in result:
                print(f"    📈 N-gram coverage: {result['coverage']:.1%}")
            
            if 'ngram_size' in result:
                print(f"    🔤 Best N-gram size: {result['ngram_size']}")
            
            # Show context if available
            if result.get('contexts'):
                print(f"    📝 Context: {result['contexts'][0]}")
        
        if len(results) > 15:
            print(f"\n... and {len(results) - 15} more documents")
        
        print("=" * 80)
        
        # Ask for document preview
        if results:
            choice = input("\n📖 Preview a document? (enter number or 'n'): ").strip()
            if choice.lower() != 'n' and choice.isdigit():
                idx = int(choice) - 1
                if 0 <= idx < len(results):
                    self.show_document_preview(results[idx]['doc_id'])
    
    def show_document_preview(self, doc_id: str, lines: int = 15):
        """Show document preview"""
        if doc_id not in self.doc_tokens:
            print(f"❌ Document not found: {doc_id}")
            return
        
        doc_info = self.documents[doc_id]
        tokens = self.doc_tokens[doc_id]
        
        print(f"\n" + "=" * 80)
        print(f"📄 DOCUMENT: {doc_info['name']}")
        print(f"📏 Tokens: {doc_info['token_count']:,}")
        print("=" * 80)
        
        # Show first N tokens
        preview_tokens = tokens[:lines*10]
        text = " ".join(preview_tokens)
        
        # Wrap text
        words = text.split()
        current_line = []
        line_length = 0
        
        print("\n" + "-" * 80)
        for word in words:
            current_line.append(word)
            line_length += len(word) + 1
            
            if line_length > 70:
                print(" ".join(current_line))
                current_line = []
                line_length = 0
        
        if current_line:
            print(" ".join(current_line))
        
        if len(tokens) > lines * 10:
            print(f"\n... and {len(tokens) - lines*10:,} more tokens")
        
        print("-" * 80)
        print("=" * 80)

def main():
    """Main function"""
    print("=" * 80)
    print("🔤 N-GRAM PHRASE SEARCH SYSTEM (up to 5-grams)")
    print("=" * 80)
    
    # Try to find corpus folder
    corpus_folder = r"C:\Users\Armaghan Rafique\Desktop\AI Project\cleaned_corpus"
    
    if not os.path.exists(corpus_folder):
        print(f"❌ Corpus folder not found: {corpus_folder}")
        
        # Try alternative
        alt_folders = [
            r"C:\Users\Armaghan Rafique\Desktop\AI Project\supreme_court_judgements_txt",
            r"C:\Users\Armaghan Rafique\Desktop\AI Project",
            os.path.join(os.path.expanduser("~"), "Desktop", "AI Project", "cleaned_corpus")
        ]
        
        found = False
        for folder in alt_folders:
            if os.path.exists(folder):
                corpus_folder = folder
                print(f"✅ Using folder: {corpus_folder}")
                found = True
                break
        
        if not found:
            corpus_folder = input("📁 Enter corpus folder path: ").strip()
            if not os.path.exists(corpus_folder):
                print(f"❌ Folder does not exist: {corpus_folder}")
                return
    
    print(f"\n📁 Corpus folder: {corpus_folder}")
    
    # Create N-gram search system with 5-grams
    ngram_system = NGramSearchSystem(corpus_folder, max_ngram_size=5)
    
    # Load or build index
    if not ngram_system.load_index():
        print("❌ Failed to initialize N-gram search system")
        return
    
    # Show welcome message
    print(f"\n🎯 N-GRAM SEARCH READY")
    print(f"   Documents: {ngram_system.stats['total_documents']:,}")
    print(f"   N-gram sizes: 2 to {ngram_system.max_ngram_size}")
    
    # Start interactive search
    ngram_system.interactive_search()

if __name__ == "__main__":
    main()

🔤 N-GRAM PHRASE SEARCH SYSTEM (up to 5-grams)

📁 Corpus folder: C:\Users\Armaghan Rafique\Desktop\AI Project\cleaned_corpus
❌ Index not found at: C:\Users\Armaghan Rafique\Desktop\AI Project\cleaned_corpus\ngram_index_5\ngram_index.pkl
Building new index...
🔨 Building N-gram Search Index (up to 5-grams)...
✅ Loaded 1460 documents
   Processing document 100/1460...
   Processing document 200/1460...
   Processing document 300/1460...
   Processing document 400/1460...
   Processing document 500/1460...
   Processing document 600/1460...
   Processing document 700/1460...
   Processing document 800/1460...
   Processing document 900/1460...
   Processing document 1000/1460...
   Processing document 1100/1460...
   Processing document 1200/1460...
   Processing document 1300/1460...
   Processing document 1400/1460...

💾 Index saved to: C:\Users\Armaghan Rafique\Desktop\AI Project\cleaned_corpus\ngram_index_5\ngram_index.pkl
📊 Statistics saved to: C:\Users\Armaghan Rafique\Desktop\AI Proj


🎯 Enter command:  murder


❌ Phrase must contain at least 2 words

🤔 No exact matches found, trying multi-size N-gram search...

❌ No results found for 'murder'

💡 Suggestions:
   • No documents found. Try:
   • 1. Check spelling of your phrase
   • 2. Try a shorter phrase
   • 3. Try individual important words



🎯 Enter command:  murder


❌ Phrase must contain at least 2 words

🤔 No exact matches found, trying multi-size N-gram search...

❌ No results found for 'murder'

💡 Suggestions:
   • No documents found. Try:
   • 1. Check spelling of your phrase
   • 2. Try a shorter phrase
   • 3. Try individual important words


...


🔍 Searching for phrase: 'availability of teachers'
   Query tokens (3): ['availability', 'of', 'teachers']
   Using 2-gram matching
   Query 2-grams (2): ['availability of', 'of teachers']
   ✗ No documents contain all 2-grams
   N-gram frequency: [0]

🤔 No exact matches found, trying multi-size N-gram search...

🔍 Multi-size N-gram search: 'availability of teachers'
   Trying 3-gram search...
   ✗ No documents found with 3-grams
   Trying 2-gram search...
   ✗ No documents found with 2-grams
   ✓ Total unique documents: 0

❌ No results found for 'availability of teachers'

💡 Suggestions:
   • No documents found. Try:
   • 1. Check spelling of your phrase
   • 2. Try a shorter phrase
   • 3. Try individual important words



🎯 Enter command:  sho ali muhammad



🔍 Searching for phrase: 'sho ali muhammad'
   Query tokens (3): ['sho', 'ali', 'muhammad']
   Using 2-gram matching
   Query 2-grams (2): ['sho ali', 'ali muhammad']
   ✗ No documents contain all 2-grams
   N-gram frequency: [0]

🤔 No exact matches found, trying multi-size N-gram search...

🔍 Multi-size N-gram search: 'sho ali muhammad'
   Trying 3-gram search...
   ✗ No documents found with 3-grams
   Trying 2-gram search...
   ✓ Found 35 documents with 2-grams
   ✓ Total unique documents: 35

✅ Search: 'sho ali muhammad'
📊 Found 35 document(s)

 1. 📄 Officer_Commanding_182_Petroleum_Storage_Platoon_A_Muhammad_Ali_vs_commanding_ofvicer.txt
    📏 Length: 839 tokens
    ⭐ Score: 33.3
    📈 N-gram coverage: 50.0%
    🔤 Best N-gram size: 2

 2. 📄 The_State_Versus_Sufi_Ali_so_Abdul_Karim_and_othe__10_._Sufi_Ali_.txt
    📏 Length: 1,803 tokens
    ⭐ Score: 33.3
    📈 N-gram coverage: 50.0%
    🔤 Best N-gram size: 2

 3. 📄 Cr_PLA_No102007_Qalb_Ali_Versus_State.txt
    📏 Length: 1,133 tokens


📖 Preview a document? (enter number or 'n'):  sho

🎯 Enter command:  muhammad


❌ Phrase must contain at least 2 words

🤔 No exact matches found, trying multi-size N-gram search...

❌ No results found for 'muhammad'

💡 Suggestions:
   • No documents found. Try:
   • 1. Check spelling of your phrase
   • 2. Try a shorter phrase
   • 3. Try individual important words
